## Imports

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 
from sklearn.svm import LinearSVC

## Data Transforming

In [8]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [9]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,1,225,308751
798459,0,6,0,0,1,0,0,0,3,0,0,0,1,690,308750
798460,0,9,0,0,2,0,0,0,3,0,0,0,1,215,308771
798461,0,16,0,0,2,0,0,0,10,0,0,0,1,219,308776


In [10]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [11]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [12]:
# c=data.subdomain_encoded.nunique()
# data.subdomain_encoded.nunique()

In [13]:
d=a+b
d

310015

In [14]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146


In [15]:

y=data['status']
X=data.drop('status', axis=1)
col=X.columns
X=StandardScaler().fit_transform(X)
X=pd.DataFrame(X,columns=col)

In [16]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=54)

In [17]:
X_train.shape

(598847, 14)

In [18]:
X_test.shape

(199616, 14)

In [19]:
X_train

,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
248801,-0.023748,1.794693,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,0.268014,-0.211449,-0.727740,-0.424454,-0.218199,-0.551873,0.868413
213106,1.943736,-0.154542,0.293937,9.410296,-0.262065,-0.143633,-0.065063,1.296563,-0.211449,1.374118,-0.424454,0.343734,-0.551873,-1.138636
556820,0.648076,2.184540,-0.363395,-0.750244,-0.262065,-0.143633,-0.065063,0.885144,-0.211449,-0.727740,-0.424454,0.343734,1.385057,0.181338
662529,-0.335666,-0.544389,-0.269491,-0.750244,0.282163,-0.143633,-0.065063,0.885144,-0.211449,-0.727740,-0.424454,0.343734,-1.073741,-1.440112
427785,1.343893,-0.154542,-0.081682,-0.072874,-0.262065,-0.143633,-0.065063,-0.554824,-0.211449,1.374118,-0.424454,0.343734,-0.551873,-1.428498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224993,-0.575604,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,-0.727740,-0.424454,-0.218199,-0.551873,-0.733869
267287,-0.239692,-0.544389,-0.363395,0.604495,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,1.374118,-0.424454,-0.218199,-0.842915,0.152235
323343,-0.143717,-0.544389,-0.363395,-0.072874,-0.262065,-0.143633,-0.065063,-0.349115,-0.211449,1.374118,-0.424454,0.343734,-0.551873,0.028666
367941,0.456126,-0.154542,-0.175586,-0.072874,-0.262065,-0.143633,-0.065063,-0.966244,-0.211449,1.374118,-0.424454,0.343734,-0.551873,1.476044


## Traditional Models

In [20]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
# trains in around 2 minutes and has 0.95 across the board

              precision    recall  f1-score   support

           0       0.95      0.94      0.95     92894
           1       0.95      0.96      0.95    106722

    accuracy                           0.95    199616
   macro avg       0.95      0.95      0.95    199616
weighted avg       0.95      0.95      0.95    199616



In [21]:
# vectorizer = TfidfVectorizer(analyzer='char')
# X=vectorizer.fit_transform(df.url)
# tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
# y1=df['status']
# X1=tfidf_df
# X_train1, X_test1, y_train1,y_test1 = train_test_split(X,y,test_size=0.2,random_state=504)

# rf = RandomForestClassifier()
# rf.fit(X_train1, y_train1)
# y_pred1 = rf.predict(X_test1)
# print(classification_report(y_test1, y_pred1))


In [22]:
clf = LinearSVC() 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
# ran for 5 minutes and did not finish training

              precision    recall  f1-score   support

           0       0.79      0.63      0.70     92894
           1       0.73      0.85      0.78    106722

    accuracy                           0.75    199616
   macro avg       0.76      0.74      0.74    199616
weighted avg       0.76      0.75      0.75    199616



In [23]:

clf = SGDClassifier() 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
# took  4 minuted but terrible results

              precision    recall  f1-score   support

           0       0.82      0.58      0.68     92894
           1       0.71      0.89      0.79    106722

    accuracy                           0.74    199616
   macro avg       0.76      0.73      0.73    199616
weighted avg       0.76      0.74      0.74    199616



# Neural Networks

In [25]:
tld_num=data['top_domain_encoded'].nunique()
ld_num=data['domain_encoded'].nunique()



## LSTM

In [26]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)


In [27]:
Lstm  = Sequential()
Lstm.add(Embedding(d,8))
Lstm.add(Dropout(0.2))
Lstm.add(LSTM(128,return_state=False))
Lstm.add(Dense(24,activation='relu'))
# Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
Lstm.add(Dense(1,activation ='sigmoid'))

In [28]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [29]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epoch when split 

Epoch 1/30


18714/18714 - 260s - 14ms/step - accuracy: 0.8538 - loss: 0.3502 - precision: 0.8224 - recall: 0.9266 - val_accuracy: 0.8746 - val_loss: 0.3116 - val_precision: 0.8390 - val_recall: 0.9472
Epoch 2/30
18714/18714 - 247s - 13ms/step - accuracy: 0.8739 - loss: 0.3117 - precision: 0.8424 - recall: 0.9400 - val_accuracy: 0.8785 - val_loss: 0.3024 - val_precision: 0.8441 - val_recall: 0.9477
Epoch 3/30
18714/18714 - 227s - 12ms/step - accuracy: 0.8762 - loss: 0.3051 - precision: 0.8443 - recall: 0.9422 - val_accuracy: 0.8797 - val_loss: 0.2991 - val_precision: 0.8471 - val_recall: 0.9456
Epoch 4/30
18714/18714 - 212s - 11ms/step - accuracy: 0.8773 - loss: 0.3021 - precision: 0.8454 - recall: 0.9431 - val_accuracy: 0.8800 - val_loss: 0.2978 - val_precision: 0.8460 - val_recall: 0.9482
Epoch 5/30
18714/18714 - 213s - 11ms/step - accuracy: 0.8783 - loss: 0.2999 - precision: 0.8460 - recall: 0.9443 - val_accuracy: 0.8799 - val_loss: 0.2970 - val_precision: 0.8457 - val_recall: 0.9483
Epoch 6/30


## CNN

In [30]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [31]:
cnn  = Sequential()
cnn.add(Embedding(d,8))
cnn.add(Dropout(0.2))
cnn.add(Conv1D(128, 5, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
# cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
cnn.add(Dense(1,activation ='sigmoid'))

In [32]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [33]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around 6 minutes an epoch when split 

Epoch 1/30


18714/18714 - 181s - 10ms/step - accuracy: 0.8653 - loss: 0.3293 - precision: 0.8339 - recall: 0.9341 - val_accuracy: 0.8735 - val_loss: 0.3097 - val_precision: 0.8321 - val_recall: 0.9563
Epoch 2/30
18714/18714 - 207s - 11ms/step - accuracy: 0.8730 - loss: 0.3137 - precision: 0.8421 - recall: 0.9385 - val_accuracy: 0.8751 - val_loss: 0.3054 - val_precision: 0.8359 - val_recall: 0.9536
Epoch 3/30
18714/18714 - 135s - 7ms/step - accuracy: 0.8746 - loss: 0.3103 - precision: 0.8437 - recall: 0.9395 - val_accuracy: 0.8765 - val_loss: 0.3037 - val_precision: 0.8395 - val_recall: 0.9506
Epoch 4/30
18714/18714 - 128s - 7ms/step - accuracy: 0.8751 - loss: 0.3083 - precision: 0.8443 - recall: 0.9397 - val_accuracy: 0.8785 - val_loss: 0.3009 - val_precision: 0.8427 - val_recall: 0.9502
Epoch 5/30
18714/18714 - 127s - 7ms/step - accuracy: 0.8757 - loss: 0.3066 - precision: 0.8447 - recall: 0.9404 - val_accuracy: 0.8783 - val_loss: 0.3005 - val_precision: 0.8431 - val_recall: 0.9490
Epoch 6/30
187